<a href="https://colab.research.google.com/github/stev3j/MachineLearning_Practice/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
sw_book1 = pd.read_csv('구지면 작은도서관 장서 대출목록 (2023년 03월).csv', encoding='cp949')
sw_book1

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자,Unnamed: 13
0,1,헤어질 결심 각본,"정서경,박찬욱 지음",을유문화사,2022,9788932474755,NaN,0,NaN,812.66,1,0,2023-03-10,NaN
1,2,화이트 러시,히가시노 게이고 지음;민경욱 옮김,㈜소미미디어,2023,9791138415477,NaN,0,NaN,833.60,1,1,2023-03-10,NaN
2,3,크리스마스 타일,김금희 지음,창비,2022,9788936438890,NaN,0,NaN,813.60,1,1,2023-03-10,NaN
3,4,체셔 크로싱,앤디 위어 글;사라 앤더슨 그림;황석희 옮김,알에이치코리아(RHK),2022,9788925577319,NaN,0,NaN,843.60,1,0,2023-03-10,NaN
4,5,자발적 방관육아,최은아 지음,쌤앤파커스,2023,9791165346805,NaN,0,NaN,598.10,1,1,2023-03-10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14775,14776,(고정관념을 날려버리는) 5분 철학 오프너,줄리아 드 퓌네스 지음;이나무 옮김,이숲,2012,9788994228150,NaN,0,NaN,104.00,1,1,2014-02-11,NaN
14776,14777,(겸손의 미덕으로 미래를 바꾼) 후진타오 이야기,박근형 지음,명진출판,2012,9788976776235,NaN,0,8,990.80,1,0,2014-02-11,NaN
14777,14778,앤디 워홀 이야기,아서 단토 지음;박선령 옮김;이혜경 엮음,명진출판,2012,9788976776280,NaN,0,10,990.80,1,0,2014-02-11,NaN
14778,14779,4001,신정아 지음,사월의책,2011,9788996461067,NaN,0,NaN,818.00,1,2,2014-02-11,NaN


In [ ]:
 # 도서명, 저자, 출판사, 발행년도가 정제되어야함
sw_book2 = sw_book1[['도서명', '저자', '출판사', '발행년도']]
sw_book2.head()
sw_book2

In [ ]:
sw_book2.update(sw_book2)
na_rows = sw_book2['도서명'].isna() | sw_book2['저자'].isna() | sw_book2['출판사'].isna() | sw_book2['발행년도'].eq(-1)
print(na_rows.sum())

In [ ]:
sw_book2

In [ ]:
sw_book3 = sw_book2.dropna(subset=['도서명', '저자', '출판사'])
sw_book3 = sw_book3[sw_book3['발행년도'] != -1]
sw_book3.head()

In [ ]:
# 도서명, 저자, 출판사, 발행년도 정제를 위한 get_book_info 함수 만들기

def get_book_info(row):
    title = row['도서명']
    author = row['저자']
    pub = row['출판사']
    year = row['발행년도']
    url = 'http://www.yes24.com/Product/Search?domain=ALL&query={}'
    r = requests.get(url.format(row['ISBN']))
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # 도서명이 null이라면 title에 a태그(클래스: gd_name)를 저장하기
    try:
        if pd.isna(title):
            title = soup.find('a', attrs={'class':'gd_name'}).get_text()
    except AttributeError:
        pass

    # 저자가 null이라면 authors에 span태그(클래스: info_auth)를 저장하기
    try:
        if pd.isna(author):
            authors = soup.find('span', attrs={'class':'info_auth'}).find_all('a')
            author_list = [auth.get_text() for auth in authors]
            author = ','.join(author_list)
    except AttributeError:
        pass

    # 출판사가 null이라면 pub에 span태그(class: info_pub)를 저장하기
    try:
        if pd.isna(pub):
            pub = soup.find('span', attrs={'class':'info_pub'}).find('a').get_text()
    except AttributeError:
        pass
    
    # 발행년도가 -1(null)이라면 year_str에 span태그(class: info_data)를 저장하기
    try:
        if year == -1:
            year_str = soup.find('span', attrs={'class':'info_date'}).get_text()
            year = re.findall(r'\d{4}', year_str)[0]
    except AttributeError:
        pass

    # title, authors, pub, year을 리턴하기
    return title, author, pub, year

In [ ]:
def data_fixing(csv_file):
    # 파일 가져오기
    sw_book1 = pd.read_csv(csv_file, encoding='cp949')

    # 도서,대출건수, 세트 ISBN 데이터 정제 하기 (형변환, 널정제)
    sw_book2 = sw_book1.astype({'도서권수':'int32', '대출건수':'int32'})
    set_isbn_na_rows = sw_book2['세트 ISBN'].isna()
    sw_book2.loc[set_isbn_na_rows, '세트 ISBN'] = ''

    # 발행년도 정제 (null -> -1)
    sw_book3 = sw_book2.replace({'발행년도':'.*(\d{4}).*'}, r'\1', regex=True)

    unkown_year = sw_book3['발행년도'].str.contains('\D', na=True)
    sw_book3.loc[unkown_year, '발행년도'] = -1

    sw_book3 = sw_book3.astype({'발행년도':'int32'})
    dangun_yy_rows = sw_book3['발행년도'].gt(4000)
    sw_book3.loc[dangun_yy_rows, '발행년도'] = sw_book3.loc[dangun_yy_rows, '발행년도'] - 2333

    dangun_year = sw_book3['발행년도'].gt(4000)
    sw_book3.loc[dangun_year, '발행년도'] = -1

    old_books = sw_book3['발행년도'].gt(0) & sw_book3['발행년도'].lt(1900)
    sw_book3.loc[old_books, '발행년도'] = -1

    na_rows = sw_book3['도서명'].isna() | sw_book3['저자'].isna() | sw_book3['출판사'].isna() | sw_book3['발행년도'].eq(-1)

    # get_book_info(도서명, 저자, 출판사, 발행년도 null 확인) 함수 적용
    sw_book4 = sw_book3[na_rows].apply(get_book_info, axis=1, result_type='expand')
    sw_book4.columns = ['도서명', '저자', '출판사', '발행년도']
    sw_book3.update(sw_book4)

    # 도서명, 저자, 출판사, 발행년도 null 정제
    sw_book6 = sw_book3.dropna(subset=['도서명','저자','출판사'])
    sw_book6 = sw_book6[sw_book6['발행년도'] != -1]

    return sw_book6

In [ ]:
sw_book6 = data_fixing('구지면 작은도서관 장서 대출목록 (2023년 03월).csv')
sw_book6.head()

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자,Unnamed: 13
0,1,헤어질 결심 각본,"정서경,박찬욱 지음",을유문화사,2022,9788932474755,,0,NaN,812.66,1,0,2023-03-10,NaN
1,2,화이트 러시,히가시노 게이고 지음;민경욱 옮김,㈜소미미디어,2023,9791138415477,,0,NaN,833.60,1,1,2023-03-10,NaN
2,3,크리스마스 타일,김금희 지음,창비,2022,9788936438890,,0,NaN,813.60,1,1,2023-03-10,NaN
3,4,체셔 크로싱,앤디 위어 글;사라 앤더슨 그림;황석희 옮김,알에이치코리아(RHK),2022,9788925577319,,0,NaN,843.60,1,0,2023-03-10,NaN
4,5,자발적 방관육아,최은아 지음,쌤앤파커스,2023,9791165346805,,0,NaN,598.10,1,1,2023-03-10,NaN
